# Lab 2: Characteristics of remotely sensed data

**Purpose:**
The purpose of this lab is to demonstrate concepts of spatial, spectral, temporal and radiometric resolution.  You will be introduced to image data from several sensors aboard various platforms.  At the completion of the lab, you will be able to understand the difference between remotely sensed datasets based on sensor characteristics and how to choose an appropriate remotely sensed dataset based on these concepts.  

**Prerequisites:** Lab 1

1. Spatial resolution

In the present context, spatial resolution means pixel size.  In practice, spatial resolution depends on the projection of the sensor's instantaneous field of view (IFOV) on the ground and how a set of radiometric measurements are resampled into a regular grid.  To see the difference in spatial resolution resulting from different sensors, visualize data at different scales from different sensors.

- MODIS.  
There are two Moderate Resolution Imaging Spectro-Radiometers ([MODIS](http://modis.gsfc.nasa.gov/)) aboard the [Terra](http://terra.nasa.gov/) and [Aqua](http://aqua.nasa.gov/) satellites.  Different MODIS [bands](http://modis.gsfc.nasa.gov/about/specifications.php) produce data at different spatial resolutions.  For the visible bands, the lowest common resolution is 500 meters (red and NIR are 250 meters).  

Search for 'MYD09GA' in the [Google EarthEngine datasets](https://developers.google.com/earth-engine/datasets/catalog/) and retrieve (copy) the path to 'MYD09GA.006 Aqua Surface Reflectance Daily Global 1km and 500m'.  Name the import myd09. ([Complete list of MODIS land products](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table).  Note that Terra MODIS datasets start with 'MOD' and MODIS Aqua datasets start with 'MYD').

In [48]:
# Initializing display and earthengine
from IPython.display import Image
%matplotlib inline

import ee
ee.Initialize()

In [49]:
myd09 = ee.ImageCollection("MODIS/006/MYD09GA")

# Define a location of interest as a point at SFO airport.
sfoPoint = ee.Geometry.Point(-122.3774, 37.6194);

#Define a region to display (from the entire planet to a small region)
area_to_display = ee.Geometry.Rectangle([-120, 40, -125, 35])

To display a false-color MODIS image, select an image acquired by the Aqua MODIS sensor and display it for SFO:

In [59]:
# Get a surface reflectance image from the MODIS MYD09GA collection.
modisImage = ee.Image(myd09.filterDate('2017-07-01').filterBounds(sfoPoint).first());

# Print the image band names to the console.
band_names = modisImage.bandNames()
print(band_names.getInfo())

['num_observations_1km', 'state_1km', 'SensorZenith', 'SensorAzimuth', 'Range', 'SolarZenith', 'SolarAzimuth', 'gflags', 'orbit_pnt', 'granule_pnt', 'num_observations_500m', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'QC_500m', 'obscov_500m', 'iobs_res', 'q_scan']


These names are described also here: [modis bands](https://modis.gsfc.nasa.gov/about/specifications.php)

In [51]:
# Use these MODIS bands for red, green, blue, respectively.
modisBands = ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'];

#// Add the image to the map.
url = modisImage.getThumbURL({'dimensions': 700,'bands':modisBands,'min': 0, 'max': 3000,'region':region_bay_area})
# print(url)
Image(url=url)

['num_observations_1km', 'state_1km', 'SensorZenith', 'SensorAzimuth', 'Range', 'SolarZenith', 'SolarAzimuth', 'gflags', 'orbit_pnt', 'granule_pnt', 'num_observations_500m', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'QC_500m', 'obscov_500m', 'iobs_res', 'q_scan']


In [52]:
# Get the scale of the data from the first band's projection:
modisScale = modisImage.select('sur_refl_b01').projection().nominalScale();

print('MODIS scale:', modisScale.getInfo());


MODIS scale: 463.3127165275


In [54]:
# Filter MSS imagery by location, date and cloudiness.
mss= ee.ImageCollection("LANDSAT/LM05/C01/T2")

mssImage = ee.Image(mss
    .filterBounds(sfoPoint)
    .sort('CLOUD_COVER')
#Get the least cloudy image.
    .first());

# Print the image band names to the console.
band_names = mssImage.bandNames()
print(band_names.getInfo())
# Display the MSS image as a color-IR composite.


['B1', 'B2', 'B3', 'B4', 'BQA']


In [55]:
# Map.addLayer(mssImage, {bands: ['B3', 'B2', 'B1'], min: 0, max: 200}, 'MSS');

# Use these MODIS bands for red, green, blue, respectively.
mssBands = ['B3', 'B2', 'B1'];

#// Add the image to the map.
url = mssImage.getThumbURL({'dimensions': 700,'bands':mssBands,'min': 0, 'max': 200,'region':region_bay_area})
# print(url)
Image(url=url)

In [58]:
region_bay_area2 = ee.Geometry.Rectangle([-122.6, 37.8, -122.3, 37.7])
#// Add the image to the map.
url = mssImage.getThumbURL({'dimensions': 700,'bands':mssBands,'min': 0, 'max': 200,'region':region_bay_area2})
# print(url)
Image(url=url)

In [57]:
# Get the scale of the MSS data from its projection:
mssScale = mssImage.select('B1').projection().nominalScale();

print('MSS scale:', mssScale.getInfo());


MSS scale: 60
